In [13]:
import pandas as pd
import numpy as np
import networkx as nx
import graph as gr
import utils as ut
import feature_selection as fs
import classification as cl
import os.path
import matplotlib.pyplot as plt
from pprint import pprint
import importlib
import community
import seaborn as sns
from gensim.models import KeyedVectors
from pyemd import emd
from gensim import corpora
from nltk.corpus import stopwords
from gensim.matutils import softcossim
import copy
import itertools
from keras.models import Model, Sequential
from keras.layers import LSTM, Dense
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Reshape
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score, roc_auc_score
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import load_model
import pickle

In [2]:
importlib.reload(gr)

<module 'graph' from 'C:\\Users\\faisal\\PycharmProjects\\linkPrediction\\graph.py'>

In [20]:
select_domain  = 0 #0:apnea ro 1:obesity
domain = ['apnea','obesity']
root = '../linkPrediction/'
filepath=[['dataset/apnea-all,3.csv',
         'dataset/apnea-distinct_keyword.csv'],
        ['dataset/obesity-all,3.csv',
         'dataset/obesity-distinct_keyword.csv']]
graphpath=root+'graphs/'+domain[select_domain]
datapath=root+'dataframes/'+domain[select_domain]
modelpath=root+'models/'+domain[select_domain]
column_split=['keyword','author_name','affiliation_1','affiliation_2','country']
#time=[parent_start_year, train_start_year, test_start_year, total_test_period_in_year, total_year_in_each_iteration]
time=[2007,2008,2015,1,1,200]
df, key_list = ut.load_dataset(filepath[select_domain], column_split)

In [25]:
for tout in range(1991,2008,8):
    time =  [tout,tout+1,tout+8,1,1,20]
    gr.dynamic_train_test_graph_build(df, key_list, graphpath, time)

272
../linkPrediction/graphs/apnea\train_graph_1992.gpickle saved, nodes: 56 edges: 41
../linkPrediction/graphs/apnea\train_graph_1993.gpickle saved, nodes: 70 edges: 56
../linkPrediction/graphs/apnea\train_graph_1994.gpickle saved, nodes: 76 edges: 59
../linkPrediction/graphs/apnea\train_graph_1995.gpickle saved, nodes: 115 edges: 147
../linkPrediction/graphs/apnea\train_graph_1996.gpickle saved, nodes: 122 edges: 149
../linkPrediction/graphs/apnea\train_graph_1997.gpickle saved, nodes: 133 edges: 129
../linkPrediction/graphs/apnea\train_graph_1998.gpickle saved, nodes: 136 edges: 145
../linkPrediction/graphs/apnea\test_graph_1999.gpickle saved, nodes: 272 edges: 302
../linkPrediction/graphs/apnea\parent_graph_1991-1992.gpickle saved, nodes: 51 edges: 55
../linkPrediction/graphs/apnea\train_graph_1992-1999.gpickle saved, nodes: 272 edges: 559
../linkPrediction/graphs/apnea\test_graph_1999-2000.gpickle saved, nodes: 272 edges: 302
539
../linkPrediction/graphs/apnea\train_graph_2000.gpi

In [30]:
graphs = {}
for tout in range(1991,2008,8):
    time =  [tout,tout+1,tout+8,1,1,20]
#     g_train, g_test, g_parent, g_train_static, g_test_static = gr.graph_load(graphpath, time)
#     graphs[tout]=g_parent
#     graphs.update(g_train)
    print(time)

[1991, 1992, 1999, 1, 1, 20]
[1999, 2000, 2007, 1, 1, 20]
[2007, 2008, 2015, 1, 1, 20]


In [10]:
k = set()
ko=set()
e = set()
eo = set()
gr_ev = []
for year,graph in graphs.items():
    k = set(graph.nodes())
    k_n = k.difference(ko)
    k_o = k.difference(k_n)
    e = set(graph.edges())
    e_ooo = e.intersection(eo)
    e_nn_no = e.difference(eo)
    e_oon = set()
    e_nn = set()
    e_no = set()
    for es in e_nn_no:
        diff = len(set(es).difference(ko))
        if diff==0:
            e_oon.add(es)
        if diff==2:
            e_nn.add(es)
        if diff==1:
            e_no.add(es)
#     print('year:',year)
#     print('k:',len(k),'kn:',len(k_n),'ko:',len(k_o))
#     print('e:',len(e),'e_ooo:',len(e_ooo),'e_oon:',len(e_oon),'e_no:',len(e_no),'e_nn:',len(e_nn))
    gr_ev.append([year,len(k),len(k_n),len(k_o),len(e),len(e_ooo),len(e_oon),len(e_no),len(e_nn)])
    ko.update(k)
    eo.update(e)
cols = ['year', 'k', 'kn', 'ko', 'e', 'e_ooo', 'e_oon', 'e_no', 'e_nn',]
gr_ke = pd.DataFrame(gr_ev, columns=cols) 

In [33]:
importlib.reload(fs)
importlib.reload(cl)
type_ev = []
for tout in range(1991,2008,8):
    time =  [tout,tout+1,tout+8,1,1,20]
    g_train, g_test, g_parent, g_train_static, g_test_static = gr.graph_load(graphpath, time)
    freq=10
    #time=[2001,2007,2015,1,1,10]
    train_data, train_data_static, parent_data, test_data_static, edge_list = cl.non_edge_feature_dataframe(g_train, 
                                                                                                             g_test,
                                                                                                             g_parent,
                                                                                                             g_train_static, 
                                                                                                             g_test_static,  
                                                                                                             time, 
                                                                                                             freq)
    node_feature, train_data = fs.dynamic_graph_feature_set(df, key_list, train_data, g_parent, g_train, g_train_static, time)
###############################################################################################################################
#     filename= domain[select_domain]+"-train_data_"+str(time[1])+"-"+str(time[2])+".pkl"
#     filename_path = os.path.join(datapath,filename)
#     with open(filename_path,"rb") as f:
#         train_data = pickle.load(f)
#         print('train data length:',len(train_data))

#     filename= domain[select_domain]+"-node_feature_"+str(time[1])+"-"+str(time[2])+".pkl"
#     filename_path = os.path.join(datapath,filename)
#     with open(filename_path,"rb") as f:
#         node_feature = pickle.load(f)
#         print('node feature length:',len(node_feature))
################################################################################################################################
    for year,node_f in node_feature.items():
        k_gp = node_f.loc[node_f['node_type_aut']==20,['node_type_aut']].count()
        k_p = node_f.loc[node_f['node_type_aut']==5,['node_type_aut']].count()
        k_c = node_f.loc[node_f['node_type_aut']==3,['node_type_aut']].count()
        k_g = node_f.loc[node_f['node_type_aut']==1,['node_type_aut']].count()
        #print('k_gp:',k_gp[0],'k_p:',k_p[0],'k_c:',k_c[0],'k_g:',k_g[0])
        e_gpgp1 = train_data[year].loc[(train_data[year]['label']==1) & (train_data[year]['type']==400),['type']].count()
        e_gpgp0 = train_data[year].loc[(train_data[year]['label']==0) & (train_data[year]['type']==400),['type']].count()
        e_gpp1 = train_data[year].loc[(train_data[year]['label']==1) & (train_data[year]['type']==100),['type']].count()
        e_gpp0 = train_data[year].loc[(train_data[year]['label']==0) & (train_data[year]['type']==100),['type']].count()
        e_gpc1 = train_data[year].loc[(train_data[year]['label']==1) & (train_data[year]['type']==60),['type']].count()
        e_gpc0 = train_data[year].loc[(train_data[year]['label']==0) & (train_data[year]['type']==60),['type']].count()
        e_gpg1 = train_data[year].loc[(train_data[year]['label']==1) & (train_data[year]['type']==20),['type']].count()
        e_gpg0 = train_data[year].loc[(train_data[year]['label']==0) & (train_data[year]['type']==20),['type']].count()
        total = train_data[year].loc[train_data[year]['label']==1,['type']].count()
        tot_gp = e_gpgp1[0] + e_gpp1[0] + e_gpc1[0]+ e_gpg1[0]
        rat = tot_gp*100/total[0]
        #print('gpgp:',e_gpgp1[0],'gpp:',e_gpp1[0],'gpc:',e_gpc1[0],'gpg',e_gpg1[0],'total:',total[0],'tot_p:',tot_gp,'rat:',rat)
        type_ev.append([year,k_gp[0],k_p[0],k_c[0],k_g[0],e_gpgp1[0],e_gpp1[0],e_gpc1[0],e_gpg1[0],tot_gp,total[0],rat])
cols = ['year','k_gp','k_p','k_c','k_g','e_gpgp1','e_gpp1','e_gpc1','e_gpg1','tot_gp','total','rat']
gr_type = pd.DataFrame(type_ev, columns=cols) 
gr_type

../linkPrediction/graphs/apnea\train_graph_1992.gpickle loaded, nodes: 56 edges: 41
../linkPrediction/graphs/apnea\train_graph_1993.gpickle loaded, nodes: 70 edges: 56
../linkPrediction/graphs/apnea\train_graph_1994.gpickle loaded, nodes: 76 edges: 59
../linkPrediction/graphs/apnea\train_graph_1995.gpickle loaded, nodes: 115 edges: 147
../linkPrediction/graphs/apnea\train_graph_1996.gpickle loaded, nodes: 122 edges: 149
../linkPrediction/graphs/apnea\train_graph_1997.gpickle loaded, nodes: 133 edges: 129
../linkPrediction/graphs/apnea\train_graph_1998.gpickle loaded, nodes: 136 edges: 145
../linkPrediction/graphs/apnea\test_graph_1999.gpickle loaded, nodes: 272 edges: 302
../linkPrediction/graphs/apnea\parent_graph_1991-1992.gpickle loaded, nodes: 51 edges: 55
../linkPrediction/graphs/apnea\train_graph_1992-1999.gpickle loaded, nodes: 272 edges: 559
../linkPrediction/graphs/apnea\test_graph_1999-2000.gpickle loaded, nodes: 272 edges: 302
pos
test_size: 217
neg
test_size: 36080
pos in t

,year,k_gp,k_p,k_c,k_g,e_gpgp1,e_gpp1,e_gpc1,e_gpg1,tot_gp,total,rat
0,1992,11,15,27,3,0,4,6,1,11,23,47.826087
1,1993,16,20,29,5,1,3,15,0,19,32,59.375000
2,1994,12,24,37,3,1,3,6,1,11,26,42.307692
3,1995,16,46,42,11,1,8,7,3,19,49,38.775510
4,1996,19,50,43,10,0,14,7,2,23,55,41.818182
5,1997,19,41,54,19,2,10,16,5,33,75,44.000000
6,1998,17,48,56,15,2,7,11,2,22,66,33.333333
7,2000,19,75,74,36,7,28,24,16,75,212,35.377358
8,2001,20,96,85,22,8,60,35,6,109,268,40.671642
9,2002,20,106,94,33,11,47,38,19,115,286,40.209790


In [29]:
importlib.reload(fs)
importlib.reload(cl)
miss_ev = []
for tout in range(1991,2008,8):
    time =  [tout,tout+1,tout+8,1,1,20]
    ts = time[1]
    te = time[2]
    g_train, g_test, g_parent, g_train_static, g_test_static = gr.graph_load(graphpath, time)
    freq=10
    #time=[2001,2007,2015,1,1,10]
    train_data, train_data_static, parent_data, test_data_static, edge_list = cl.non_edge_feature_dataframe(g_train, 
                                                                                                             g_test,
                                                                                                             g_parent,
                                                                                                             g_train_static, 
                                                                                                             g_test_static,  
                                                                                                             time, 
                                                                                                             freq)
    node_feature, train_data = fs.dynamic_graph_feature_set(df, key_list, train_data, g_parent, g_train, g_train_static, time)

###############################################################################################################################
#     filename= domain[select_domain]+"-train_data_"+str(time[1])+"-"+str(time[2])+".pkl"
#     filename_path = os.path.join(datapath,filename)
#     with open(filename_path,"rb") as f:
#         train_data = pickle.load(f)
#         print('train data length:',len(train_data))

#     filename= domain[select_domain]+"-node_feature_"+str(time[1])+"-"+str(time[2])+".pkl"
#     filename_path = os.path.join(datapath,filename)
#     with open(filename_path,"rb") as f:
#         node_feature = pickle.load(f)
#         print('node feature length:',len(node_feature))
################################################################################################################################
    static_pos = set(train_data_static[(train_data_static['label']==1)]['row_name'])
    dynamic_pos = set()
    for t in range(ts,te,1):
        dynamic_pos.update(train_data[t][(train_data[t]['label']==1)]['row_name'])
    transitory_edges = static_pos.difference(dynamic_pos)
    transitory_nodes = set()
    for row in transitory_edges:
        transitory_nodes.add(row[0])
        transitory_nodes.add(row[1]) 
    nodes_te =set()
    nodes_te1 = set()
    nodes_teo = set()
    for node in transitory_nodes:
        if (te-1) in set(g_train_static.nodes[node]['year']):
            nodes_te.add(node)
        if (te-2) in set(g_train_static.nodes[node]['year']):
            nodes_te1.add(node)
    nodes_te1 = nodes_te1.difference(nodes_te)        
    nodes_teo = transitory_nodes.difference(nodes_te,nodes_te1)
    #print('n_te:',len(nodes_te),'n_te1:',len(nodes_te1),'n_teo:',len(nodes_teo),'n_tot:',len(transitory_nodes))
    e_tete1= set()
    e_teteo= set()
    for row in transitory_edges:
        diff = len(set(row).difference(nodes_te))
        if diff==1:
            diff1 = len(set(row).difference(nodes_te1))
            if diff1 ==1:
                e_tete1.add(row)
            if diff1 ==2:
                e_teteo.add(row)
    tot_te = len(e_tete1)+len(e_teteo)
    rat = tot_te*100/len(transitory_edges)
    #print('e_tete1:',len(e_tete1),'e_teteo:',len(e_teteo),'tot_e:',tot_te,'total:',len(transitory_edges),'rat:',rat) 
    miss_ev.append([ts,te-1,len(nodes_te),len(nodes_te1),len(nodes_teo),len(transitory_nodes),
                    len(e_tete1),len(e_teteo),tot_te,len(transitory_edges),rat])
cols = ['ts','te','n_te','n_te1','n_teo','n_trans','e_tete1','e_teteo','tot_te','tot_e_trans','rat']
gr_miss=pd.DataFrame(miss_ev, columns=cols)
gr_miss

../linkPrediction/graphs/apnea\train_graph_1992.gpickle loaded, nodes: 56 edges: 41
../linkPrediction/graphs/apnea\train_graph_1993.gpickle loaded, nodes: 70 edges: 56
../linkPrediction/graphs/apnea\train_graph_1994.gpickle loaded, nodes: 76 edges: 59
../linkPrediction/graphs/apnea\train_graph_1995.gpickle loaded, nodes: 115 edges: 147
../linkPrediction/graphs/apnea\train_graph_1996.gpickle loaded, nodes: 122 edges: 149
../linkPrediction/graphs/apnea\train_graph_1997.gpickle loaded, nodes: 133 edges: 129
../linkPrediction/graphs/apnea\train_graph_1998.gpickle loaded, nodes: 136 edges: 145
../linkPrediction/graphs/apnea\test_graph_1999.gpickle loaded, nodes: 272 edges: 302
../linkPrediction/graphs/apnea\parent_graph_1991-1992.gpickle loaded, nodes: 51 edges: 55
../linkPrediction/graphs/apnea\train_graph_1992-1999.gpickle loaded, nodes: 272 edges: 559
../linkPrediction/graphs/apnea\test_graph_1999-2000.gpickle loaded, nodes: 272 edges: 302
pos
test_size: 217
neg
test_size: 36080
pos in t

,ts,te,n_te,n_te1,n_teo,n_trans,e_tete1,e_teteo,tot_te,tot_e_trans,rat
0,1992,1998,24,18,39,81,8,20,28,52,53.846154
1,2000,2006,17,17,126,160,6,17,23,118,19.491525
2,2008,2014,73,53,93,219,31,53,84,130,64.615385


In [212]:
# gr_type
# print(gr_type)

In [211]:
# gr_miss
#print(gr_miss.to_latex(float_format=lambda x: '%.2f' % x))

In [208]:
#gr_ke
#print(gr_ke.to_latex(float_format=lambda x: '%.2f' % x))

In [210]:
#df.groupby('art_year').count()
# cols = ['year','total article']
# obesity_stat=pd.DataFrame(d2, columns=cols)
# print(obesity_stat.to_latex(float_format=lambda x: '%.2f' % x))

In [11]:
gr_ke

,year,k,kn,ko,e,e_ooo,e_oon,e_no,e_nn
0,2007,2108,2108,0,9577,0,0,0,9577
1,2008,2357,784,1573,10285,1836,6036,2283,130
2,2009,2092,242,1850,10866,2850,6550,1407,59
3,2010,2301,166,2135,12449,3842,8064,536,7
4,2011,2402,80,2322,14811,5332,9199,279,1
5,2012,623,60,563,1304,758,305,233,8
6,2013,707,22,685,1406,890,462,54,0
7,2014,3218,563,2655,18244,4640,11204,2302,98


In [15]:
gr_type

,year,k_gp,k_p,k_c,k_g,e_gpgp1,e_gpp1,e_gpc1,e_gpg1,tot_gp,total,rat
0,2008,20,1765,141,431,1,768,75,211,1055,6720,15.699405
1,2009,20,1623,93,356,0,744,31,198,973,6379,15.253174
2,2010,20,1758,114,409,0,779,50,196,1025,7071,14.495828
3,2011,20,1966,87,329,0,914,38,165,1117,7502,14.889363
4,2012,14,332,135,142,0,140,67,79,286,1536,18.619792
5,2013,20,364,168,155,2,150,63,67,282,1697,16.617560
6,2014,20,2185,156,857,7,946,69,369,1391,11626,11.964562


In [35]:
print(gr_ke.to_latex(float_format=lambda x: '%.2f' % x))

\begin{tabular}{lrrrrrrrrr}
\toprule
{} &  year &     k &    kn &    ko &      e &  e\_ooo &  e\_oon &  e\_no &  e\_nn \\
\midrule
0 &  2007 &  2108 &  2108 &     0 &   9577 &      0 &      0 &     0 &  9577 \\
1 &  2008 &  2357 &   784 &  1573 &  10285 &   1836 &   6036 &  2283 &   130 \\
2 &  2009 &  2092 &   242 &  1850 &  10866 &   2850 &   6550 &  1407 &    59 \\
3 &  2010 &  2301 &   166 &  2135 &  12449 &   3842 &   8064 &   536 &     7 \\
4 &  2011 &  2402 &    80 &  2322 &  14811 &   5332 &   9199 &   279 &     1 \\
5 &  2012 &   623 &    60 &   563 &   1304 &    758 &    305 &   233 &     8 \\
6 &  2013 &   707 &    22 &   685 &   1406 &    890 &    462 &    54 &     0 \\
7 &  2014 &  3218 &   563 &  2655 &  18244 &   4640 &  11204 &  2302 &    98 \\
\bottomrule
\end{tabular}

